In [1]:
from icalendar import Calendar, Event
import pandas as pd

In [2]:
ical_file = 'schedule.ics'
with open(ical_file) as f:
    ical_text = f.read()
cal = Calendar.from_ical(ical_text)
w = cal.walk()
w = w[1:] # cal is the first item for some reason

In [3]:
def event_to_dict(e):
    event_dict = {key.lower() : clean_bytes(e.decoded(key)) for key in e}
    del event_dict['uid']
    del event_dict['sequence']
    desc_dict = extract_desc(event_dict['description'])
    for key in desc_dict:
        event_dict[key] = desc_dict[key]
    del event_dict['description']
    
    event_dict['person'] = extract_person(event_dict['summary'], event_dict['shift'])

    return event_dict

def extract_person(summary_str, shift_code):
    summ_str_split = summary_str.split(' ')
    return summ_str_split[-2] + ' ' + summ_str_split[-1]
    # return summary_str.split(shift_code + ' ')[-1]

def extract_desc(desc):
    lines = desc.split('\n')
    desc_dict = {}
    for line in lines:
        colon_loc = line.find(':')
        key = line[:colon_loc]
        val = line[colon_loc+1:]
        desc_dict[key.lower()] = val[1:]
    return desc_dict

def clean_bytes(b):
    if type(b) == bytes:
        return b.decode('utf8')
    else:
        return b

In [4]:
df = pd.DataFrame([event_to_dict(e) for e in w]).sort_values(['dtstart','facility','shift'])

In [32]:
def get_shift_level(s : str):
    '''Returns the minimum PGY level to work the shift'''
    if (s.find('EC3') >= 0) or (s.find('Jeopardy') >= 0):
        return 2
    if (s.find('UP') >= 0) or (s.find('HP') >= 0):
        return 3
    elif s in {'SH','SI','UH','UI'}:
        return 1
    elif s in {'SA','SB','SD','SE','UC','UG','UK','UR','UF'}:
        return 2
    elif s in {'UX','UY','UZ'}:
        return 3
    elif s in {'SL','SJ','UL','UJ'}:
        return 4
    else:
        return 0 # not a tradable shift

def make_shift_df(df : pd.DataFrame):
    df = df.copy()
    shift_df = df[['shift','dtstart','dtend','shift type','facility']]
    shift_df['length'] = (shift_df.dtend - shift_df.dtstart).dt.seconds / 60 / 60
    shift_df.loc[shift_df['shift'] == 'Jeopardy AM','length'] = 12
    shift_df['start'] = shift_df.dtstart.dt.hour 
    shift_df['end'] = shift_df.dtend.dt.hour
    shift_df.loc[shift_df['shift'] == 'Jeopardy AM','end'] = 0
    shift_df = shift_df.drop(['dtstart','dtend'], axis=1)
    shift_df = shift_df.drop_duplicates('shift').sort_values('shift')
    shift_df['level'] = shift_df['shift'].apply(get_shift_level)

    return shift_df

In [33]:
# A list of shifts that can't be traded into
UNTRADABLES = ['EMS MICU Night', 'Survival Flight', 'Teaching Shift', 'Trauma Team Training']

In [34]:
roster = pd.read_csv('resident_roster.csv')
shifts = make_shift_df(df)

/tmp/ipykernel_5254/977539234.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shift_df['length'] = (shift_df.dtend - shift_df.dtstart).dt.seconds / 60 / 60
/home/maxspad/anaconda3/envs/trades/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/tmp/ipykernel_5254/977539234.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [35]:
roster

,person,pgy
0,A Hall,3
1,A Kadri,4
2,A Krumheuer,2
3,A Miller,3
4,A Rimawi,1
...,...,...
59,S Stringer,4
60,S Vermillion,3
61,V Shahi,3
62,W Sturdavant,3


In [36]:
shifts

,shift,shift type,facility,length,start,end,level
9,11:00am-11:00pm EC3,"EM 2,3,4 (EC3 Shifts)",University of Michigan,12.0,11,23,2
23,11:00pm-8:00am EC3 Night,"EM 2,3,4 (EC3 Shifts)",University of Michigan,9.0,23,8,2
141,8:00am-8:00pm EC3,"EM 2,3,4 (EC3 Shifts)",University of Michigan,12.0,8,20,2
19,8:00pm-5:00am EC3 Night,"EM 2,3,4 (EC3 Shifts)",University of Michigan,9.0,20,5,2
1341,EMS MICU Night,Track - EMS,University of Michigan,12.0,17,5,0
6,Jeopardy AM,Jeopardy,University of Michigan,12.0,7,0,2
18,Jeopardy PM,Jeopardy,University of Michigan,12.0,19,7,2
0,SA,"EM 23 (SA, SB, SD)",St. Joseph Mercy Hospital,10.0,6,16,2
12,SB,"EM 23 (SA, SB, SD)",St. Joseph Mercy Hospital,10.0,14,0,2
21,SD,"EM 23 (SA, SB, SD)",St. Joseph Mercy Hospital,10.0,22,8,2
